In [1]:
param_date = '20240706'

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [3]:
input_ = pd.read_excel(f'./{param_date}/6.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Type No'] = input_['Type No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Type'].tolist()

总数量：2272


[' Air Suspension Line Repair Kit',
 '110 Volt Accessory Power Outlet',
 '12 Volt Accessory Power Outlet Cover',
 '12 Volt Accessory Power Outlet Socket',
 '4 Wheel Drive Switch',
 '4WD Actuator Cable',
 '4WD Actuator Fork',
 '4WD Actuator Valve',
 '4WD Actuator Wiring Harness',
 '4WD Axle Actuator',
 '4WD Axle Actuator Housing',
 '4WD Disconnect',
 '4WD Disconnect Gear',
 '4WD Disconnect Gear Kit',
 '4WD Hub Locking Actuator Delete',
 '4WD Hub Locking Actuator Vacuum Hose',
 '4WD Hub Locking Solenoid',
 '4WD Vacuum Check Valve',
 'A/C Accumulator Bracket Retainer',
 'A/C Compressor Clutch',
 'A/C Compressor Clutch Dust Cover Retaining Clip',
 'A/C Compressor Clutch Relay',
 'A/C Compressor Control Valve',
 'A/C Compressor Flow Sensor',
 'A/C Condenser Bolt',
 'A/C Condenser Fan Assembly',
 'A/C Condenser Fan Motor Relay',
 'A/C Expansion Valve Mount Plate',
 'A/C Hose Assembly',
 'A/C Hose Bolt',
 'A/C Line Clip',
 'A/C Refrigerant Pressure Sensor',
 'A/C Service Valve Cap',
 'A/C Vac

In [4]:
a = 0

a

0

In [5]:
crawler_status = 'error'

crawler_status

'error'

In [6]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

resp = requests.get(input_.loc[a, 'Type Url'],
                    headers=get_header(),
                    proxies=get_proxy(),
                    timeout=(10, 10)).text

resp

'\r\n<!doctype html>\r\n<html lang="en">\r\n<head>\r\n    <title>Dorman Products - Search Results</title>\r\n\t<meta charset="utf-8" />\r\n    <meta http-equiv="cache-control" content="no-cache, no-store, must-revalidate" />\r\n    <meta http-equiv="pragma" content="no-cache" />\r\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n    <meta http-equiv="expires" content="-1" />\r\n\r\n    <meta name="description" content="Dorman gives auto repair professionals and vehicle owners greater freedom to fix cars and trucks by focusing on solutions first." />\r\n    <meta name="keywords" content="auto parts, car parts, automotive parts, aftermarket car parts, automotive repair parts, window regulator, control arm, door handle, fluid reservoir, remanufactured auto parts, intake manifold, exhaust manifold, diesel automotive part, instrument cluster, automotive electronic parts, clock spring, engine car part, chassis, car part, car mirror, transmission parts" />\r\n    

In [7]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(resp.strip(), 'lxml')
html = etree.HTML(str(soup))

with open('./page.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Dorman Products - Search Results
  </title>
  <meta charset="utf-8"/>
  <meta content="no-cache, no-store, must-revalidate" http-equiv="cache-control"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="-1" http-equiv="expires"/>
  <meta content="Dorman gives auto repair professionals and vehicle owners greater freedom to fix cars and trucks by focusing on solutions first." name="description"/>
  <meta content="auto parts, car parts, automotive parts, aftermarket car parts, automotive repair parts, window regulator, control arm, door handle, fluid reservoir, remanufactured auto parts, intake manifold, exhaust manifold, diesel automotive part, instrument cluster, automotive electronic parts, clock spring, engine car part, chassis, car part, car mirror, transmission parts" name="keywords"/>
  <meta content="width=device-width, initial-scale=1.0, max

In [8]:
count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())

count

7

In [9]:
count <= 0

False

In [10]:
import math

df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                         'Type': input_.loc[a, 'Type'],
                         'Type Url': input_.loc[a, 'Type Url'],
                         'SKU Count': count,
                         'Page': math.ceil(count / 100)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,Type No,Type,Type Url,SKU Count,Page
0,1,Air Suspension Line Repair Kit,https://www.dormanproducts.com/gsearch.aspx?ty...,7,1


In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                             'Type': input_.loc[a, 'Type'],
                             'Type Url': input_.loc[a, 'Type Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [13]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_page.xlsx', index=False)

output_correct

,Type No,Type,Type Url,SKU Count,Page
0,1,Air Suspension Line Repair Kit,https://www.dormanproducts.com/gsearch.aspx?ty...,7,1


In [14]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['Type No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_page_error.xlsx', index=False)

output_error

""
